In [3]:
import torch
from transformers import BertConfig, BertForMaskedLM, AutoTokenizer
import torch.nn.functional as F

In [4]:
def token_to_ids(sentence_input):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentence_input))

In [5]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased") 

epoch = "0"
path = "bert_trained_model_regular/" + epoch +"/"
config = BertConfig.from_pretrained( path + "config.json" )
model = BertForMaskedLM.from_pretrained( path + "pytorch_model.bin", config=config)

# setting device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
if  torch.cuda.device_count()>1:         
    model = torch.nn.DataParallel(model,device_ids=[0,1])
model.to(device)
model.eval()

DataParallel(
  (module): BertForMaskedLM(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
           

In [6]:
# obj = {"obs1": "Chad went to get the wheel alignment measured on his car.", "obs2": "The mechanic provided a working alignment with new body work.", "hyp_plus": "Chad was waiting for his car to be finished.", "hyp_minus": "Chad was waiting for his car to be washed."}
# obj = {"obs1": "Chad loves Barry Bonds.", "obs2": "Chad ensured that he took a picture to remember the event.", "hyp_plus": "Chad went to meet Barry Bonds.", "hyp_minus": "Chad failed to meet Barry Bonds."}
# obj = {"obs1": "Homer bought a gas grill for the summer.", "obs2": "They grilled steak for the first time on the grill.", "hyp_plus": "He wanted a steak.", "hyp_minus": "He did not like eating steaks."}
obj = {"obs1": "Brenna and I used to be best friends.", "obs2": "We never talked again.", "hyp_plus": "Brenna and I fought over a boy.", "hyp_minus": "Breanna and I went to the mall together."}
# obj = {"obs1": "My friend is a hunter.", "obs2": "The elk was nowhere to be found.", "hyp_plus": "She set up a hunting blind in the woods.", "hyp_minus": "Breanna and I went to the mall together."}


obs1 = '[CLS]' + obj['obs1'] + '[SEP]'
obs2 = obj['obs2'] + '[SEP]'

obs1_word_piece = tokenizer.tokenize(obs1)
obs2_word_piece = tokenizer.tokenize(obs2)

obs1_ids = tokenizer.convert_tokens_to_ids(obs1_word_piece)
obs2_ids = tokenizer.convert_tokens_to_ids(obs2_word_piece)

In [10]:
input_ids = []
token_type_ids = []
attention_mask = []

# 先append第一句 token_type為0
for i in range(len(obs1_ids)):
    input_ids.append(obs1_ids[i])
    token_type_ids.append(0)
    attention_mask.append(1)

# append[MASK] token_type為1
count = 0
hyp = ""
maskpos = len(input_ids)

input_ids.append(103)
token_type_ids.append(1)
attention_mask.append(1)

# append第三句 token_type為0
for i in range(len(obs2_ids)):
    input_ids.append(obs2_ids[i])
    token_type_ids.append(0)
    attention_mask.append(1)

    
#補齊長度為512
while len(input_ids)<512:
    input_ids.append(0)
    token_type_ids.append(0)
    attention_mask.append(0)

while count < 512:
    input_ids_tensor = torch.LongTensor([input_ids])
    token_type_ids_tensor = torch.LongTensor([token_type_ids])
    attention_mask_tensor = torch.LongTensor([attention_mask])
    
    output = model(
        input_ids = input_ids_tensor,
        token_type_ids = token_type_ids_tensor, 
        attention_mask = attention_mask_tensor
    )
    predicts = output[0]
    predicts_index = torch.argmax(predicts[0, maskpos]).item()
    
    
#     if torch.topk(predicts[0, maskpos],3,dim=-1).indices[0].item() == 102:
#         predicts_index = torch.topk(predicts[0, maskpos],3,dim=-1).indices[1].item()
#     else:
#         predicts_index = torch.topk(predicts[0, maskpos],3,dim=-1).indices[1].item()
    
   
    
    
    predicts_token = tokenizer.convert_ids_to_tokens(predicts_index)
    
    # --------------
#     decode_token = tokenizer.decode(predicts_index)
#     print(predicts_token, decode_token)
    #-------------
    
    if predicts_token == "[SEP]":
        break

    elif predicts_token.startswith("##"):
        hyp = hyp + predicts_token[2:]
    else:
        hyp = hyp + " " + predicts_token

    input_ids[maskpos] = predicts_index
    token_type_ids[maskpos] = 1
    attention_mask[maskpos] = 1
    
    maskpos += 1
    if maskpos < 512:
        input_ids[maskpos] = 103
    else:
        break
    count += 1

print(obs1 + obs2)
print("answer : " + obj['hyp_plus'])
print("predict :" + hyp)
        

(tensor([[[ -6.0291,  -5.9745,  -5.9655,  ...,  -5.4773,  -5.4335,  -3.4958],
         [-10.5945, -10.7573, -10.7030,  ...,  -9.9779,  -9.1868,  -8.8025],
         [ -6.9924,  -7.2248,  -6.9725,  ...,  -6.4492,  -6.4662,  -7.7672],
         ...,
         [-12.1683, -12.0564, -11.9398,  ..., -11.5944, -11.2568, -12.6472],
         [-11.9891, -11.9074, -11.7754,  ..., -11.4530, -11.1173, -12.4273],
         [-11.9553, -11.8534, -11.7369,  ..., -11.4282, -11.0813, -12.5840]]],
       device='cuda:0', grad_fn=<GatherBackward>),)
(tensor([[[ -8.6216,  -8.5494,  -8.4935,  ...,  -7.8683,  -7.5583,  -4.7854],
         [-11.4656, -11.5221, -11.2541,  ..., -10.7798,  -9.3200,  -8.3493],
         [ -6.4423,  -6.7222,  -6.3886,  ...,  -5.9790,  -5.9642,  -7.8360],
         ...,
         [-13.2852, -13.1841, -13.1890,  ..., -12.9669, -12.6817, -13.0162],
         [-13.5074, -13.3998, -13.3807,  ..., -13.1408, -12.9098, -13.4225],
         [-10.9617, -10.9643, -10.9656,  ..., -10.7707, -10.5591, -10